<h1><center> Deep Melanoma Classifier</center></h1>


[Melanoma](https://www.mayoclinic.org/diseases-conditions/melanoma/symptoms-causes/syc-20374884) is the most serious type of skin cancer, develops in the cells (melanocytes) that produce melanin, early diagnosis is key for recovery.

-  The model in this app is based on the EfficientNet B5 architecture, trained on the [SIIM](https://siim.org/) Melanoma dataset.
- The model achieves SOTA performance of 0.9339 AUROC on the test data utilizing heavy [Test Time Augmentation](https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d) (T.T.A.), 55 to be exact. 
- This web app allows the user to set the number of T.T.A.s

**NOTE** This is a proof of concept for research purposes and the result obatined here should not be used as FINAL diagnosis. Always seek professional medical help in a clinical setting for final diagnosis.
    
    
<h3><center>Directions For Use:</center></h3>   

    
1. Select the desired # of Test Time Augmentations
2. Upload an image of the skin lesion you want to check for Melanoma
3. You will see in realtime, the images of test time augmentations and the corresponding model prediction.
4. Final prediction (avergae of all TTA predictions) is displayed along with the original image uploaded at the end.    

**Note:** Model Prediction is the Probability of skin legion being Melanoma

**Note:** The binder docker is a little slow, be patient with the App runtime.

In [1]:
## import the req libraries
import numpy as np
import tensorflow as tf
import efficientnet.tfkeras as efn
import ipywidgets as widgets
from PIL import Image
import h5py
import io
from ipywidgets import VBox
from tensorflow.keras.preprocessing.image import apply_affine_transform
import time
from tensorflow.keras.models import model_from_json

In [2]:
## download model weights
! python download_gdrive.py 1Glog8ezbSN-cbZ49UZsL1C9U1AUr_5P0 EfficientNetB5-weights.15.hdf5

In [3]:
## load the model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
## load weights into model
model.load_weights('EfficientNetB5-weights.15.hdf5')

In [4]:
##upload button
btn_upload = widgets.FileUpload(accept ='image/*'
                                , multiple = True
                                , align_items='center')

In [5]:
## slider for tta
sldr = widgets.IntSlider(value=10
                         , min=5
                         , max=50
                         , step=1
                         , description='T.T.A.:'
                         , disabled=False
                         , continuous_update=False
                         , orientation='horizontal'
                         , align_items='center'
                         , readout=True
                         , readout_format='d')

In [6]:
out = widgets.Output()
lbl_pred = widgets.Label()

In [7]:
## the onclick event for upload button
def on_click(change):
    img = widgets.Image(value = btn_upload.data[-1])
    out.clear_output()
    with out: display(img)
    image1 = Image.open(io.BytesIO(btn_upload.data[-1]))
    image = np.array(image1)
    pred_ls =[]
    for i in range(sldr.value):
        ## convert to tensor 
        image = tf.convert_to_tensor(image, tf.float32)
        image = tf.image.resize(image, [256,256])
        image = np.array(image)

        ## translation
        transformation = apply_affine_transform(image
                                                , tx = (np.random.normal(0, 1, 1)[0])*6
                                                , ty = (np.random.normal(0, 1, 1)[0])*6)
        ## rotation 
        transformation = apply_affine_transform(transformation
                                                , theta = 180 * (np.random.normal(0, 1, 1)[0]))
        ## zoom
        transformation = apply_affine_transform(transformation
                                                , zx = 1 + (np.random.normal(0, 1, 1)[0])/6
                                                , zy = 1 + (np.random.normal(0, 1, 1)[0])/6)
        ## shear
        transformation = apply_affine_transform(transformation
                                                , shear = (np.random.normal(0, 1, 1)[0])*1.5
                                                )
        img1 = tf.image.random_flip_left_right(transformation)
        img1 = tf.image.random_hue(img1, 0.01)
        img1 = tf.image.random_saturation(img1, 0.7, 1.3)
        #img1 = tf.image.random_contrast(img1, 0.8, 1.2)
        img1 = tf.image.random_brightness(img1, 0.1)
        img1 = tf.image.random_crop(img1, [250, 250, 3])

        img1 = tf.cast(img1, tf.float32) / 255.0
        input_arr = tf.image.resize(img1, [248,248])
        input_arr = tf.reshape(input_arr, [248,248, 3])
        out.clear_output()
        img2 = Image.fromarray(((np.array(input_arr)*255).astype(np.int8)), 'RGB')
        with out: display(img2)
        input_arr = tf.expand_dims(input_arr, axis=0)
        pred = model.predict(input_arr, batch_size=1)[0][0]
        pred_ls.append(pred)
        lbl_pred.value = f'T.T.A. # {i+1} Prediction :{ str(round(pred,2))}'
        # Wait for 5 seconds
        time.sleep(5)
    
    out.clear_output()
    img = image1.resize((248,248))
    with out: display(img)
    ans = str(round(np.mean(pred_ls),2))    
    lbl_pred.value = f'Final Probability of Melanoma for the uploaded(above) image { ans}'
    

In [8]:
btn_upload.observe(on_click, names=['data'])

In [9]:
##organize widgets
display(VBox([widgets.Label('Select the # Test Time Augmentations ( T.T.A. )')
              , sldr
              , widgets.Label('Select your image')
              , btn_upload
              , out
              ,lbl_pred]))

##### PhaleoHealth